In [1]:
import os
import numpy as np
import pandas as pd
import hickle as hkl

from collections import defaultdict

from sklearn.metrics import roc_auc_score, r2_score, accuracy_score, explained_variance_score, mean_absolute_error, mean_squared_error

import keras
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense, MultiHeadAttention, Add, GlobalAveragePooling1D, Conv1D, LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from IPython.core.display import display, HTML



# Hide GPU from visible devices

#'''
tf.config.set_visible_devices([], 'GPU')
print(f'CUDA GPU AVAILABLE: {tf.test.is_gpu_available(cuda_only=True)}')
'''
THREADS = 8
os.environ['OMP_NUM_THREADS'] = str(THREADS)
os.environ['TF_NUM_INTEROP_THREADS'] = str(THREADS)
os.environ['TF_NUM_INTRAOP_THREADS'] = str(THREADS)
tf.config.threading.set_inter_op_parallelism_threads(THREADS)
tf.config.threading.set_intra_op_parallelism_threads(THREADS)
tf.config.set_soft_device_placement(True)
'''

#'''
display(HTML("<style>.container { width:100% !important; }</style>"))
#'''

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
CUDA GPU AVAILABLE: True


In [2]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean((y_pred - y_true) / y_true)

In [3]:
N_SPLITS = 5
N_STEPS = 20
N_DAYS = 60
DATA_DIR = f'data/processed_data/cv_data__{N_SPLITS}_class_id_folds__{N_STEPS}_steps__{N_DAYS}_days'

In [4]:
results = []

for i in range(N_SPLITS):

    # Load one partition of folds
    data = hkl.load(f'{DATA_DIR}/fold_{i}.hkl')
    action_training_input = data['action_training_input']
    recurrent_training_input = data['recurrent_training_input']
    prior_training_input = data['prior_training_input']
    completion_training_target = data['completion_training_target']
    problems_training_target = data['problems_training_target']
    training_target_sequence = data['training_target_sequence']
    action_testing_input = data['action_testing_input']
    recurrent_testing_input = data['recurrent_testing_input']
    prior_testing_input = data['prior_testing_input']
    completion_testing_target = data['completion_testing_target']
    problems_testing_target = data['problems_testing_target']
    testing_target_sequence = data['testing_target_sequence']
    
    # Clear session so models don't pile up
    keras.backend.clear_session()
    
    # Recurrent Input
    recurrent_input_layer = Input(shape=recurrent_training_input[0].shape, name='recurrent')
    
    x = LayerNormalization(epsilon=1e-6)(recurrent_input_layer)
    x = MultiHeadAttention(key_dim=128, num_heads=4, dropout=0.5)(x, x)
    x = Dropout(0.5)(x)
    res = Add()([x, recurrent_input_layer])
    x = LayerNormalization(epsilon=1e-6)(res)
    x = Conv1D(filters=4, kernel_size=1, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Conv1D(filters=recurrent_training_input.shape[-1], kernel_size=1)(x)
    x = Add()([x, res])
    
    x = LayerNormalization(epsilon=1e-6)(x)
    x = MultiHeadAttention(key_dim=128, num_heads=4, dropout=0.5)(x, x)
    x = Dropout(0.5)(x)
    res = Add()([x, recurrent_input_layer])
    x = LayerNormalization(epsilon=1e-6)(res)
    x = Conv1D(filters=4, kernel_size=1, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Conv1D(filters=recurrent_training_input.shape[-1], kernel_size=1)(x)
    x = Add()([x, res])
    
    x = GlobalAveragePooling1D(data_format='channels_first')(x)
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    completion_output_layer = Dense(units=1, activation='sigmoid', name='completion')(x)
    problems_output_layer = Dense(units=1, activation='linear', name='problems')(x)
    
    combined_model = Model(recurrent_input_layer, [completion_output_layer, problems_output_layer])
    combined_model.compile(optimizer=Adam(learning_rate=1e-4), loss={'completion': 'binary_crossentropy', 'problems': 'mse'})

    # Train model
    es = [EarlyStopping(monitor='val_loss', patience=10, min_delta=0, restore_best_weights=True)]
    weights = {'completion': np.ones_like(completion_training_target) * 16, 'problems': completion_training_target}
    combined_model.fit(x={'recurrent': recurrent_training_input},
                       y={'completion': completion_training_target, 'problems': problems_training_target},
                       epochs=1000,
                       validation_split=0.25,
                       callbacks=es,
                       sample_weight=weights,
                       verbose=1)
    
    # Store model predictions
    completion_testing_output, problems_testing_output = combined_model.predict({'recurrent': recurrent_testing_input})
    df = pd.DataFrame(zip(np.ones_like(testing_target_sequence).flatten() * i, 
                          testing_target_sequence.flatten(), 
                          completion_testing_target.flatten(), 
                          problems_testing_target.flatten(), 
                          completion_testing_output.flatten(), 
                          problems_testing_output.flatten()), 
                      columns = ['fold', 
                                 'target_sequence', 
                                 'completion_target', 
                                 'problems_target', 
                                 'completion_prediction', 
                                 'problems_prediction'])
    results.append(df)
    

pd.concat(results).to_csv('attention_cross_validation_results.csv', index=False)

Epoch 1/1000
3949/3949 [==============================] - 349s 88ms/step - loss: 18.6895 - completion_loss: 9.3759 - problems_loss: 9.3136 - val_loss: 15.7741 - val_completion_loss: 9.4778 - val_problems_loss: 6.2963
Epoch 2/1000
3949/3949 [==============================] - 407s 103ms/step - loss: 16.7419 - completion_loss: 8.7198 - problems_loss: 8.0222 - val_loss: 15.6594 - val_completion_loss: 9.2904 - val_problems_loss: 6.3691
Epoch 3/1000
3949/3949 [==============================] - 367s 93ms/step - loss: 16.0357 - completion_loss: 8.3594 - problems_loss: 7.6762 - val_loss: 15.1178 - val_completion_loss: 8.7469 - val_problems_loss: 6.3709
Epoch 4/1000
3949/3949 [==============================] - 438s 111ms/step - loss: 15.1510 - completion_loss: 7.7940 - problems_loss: 7.3570 - val_loss: 14.3758 - val_completion_loss: 8.5439 - val_problems_loss: 5.8319
Epoch 5/1000
3949/3949 [==============================] - 414s 105ms/step - loss: 14.8850 - completion_loss: 7.6495 - problems_los

3949/3949 [==============================] - 302s 77ms/step - loss: 14.0731 - completion_loss: 7.5445 - problems_loss: 6.5286 - val_loss: 14.4448 - val_completion_loss: 8.5030 - val_problems_loss: 5.9418
Epoch 8/1000
3949/3949 [==============================] - 302s 77ms/step - loss: 13.9837 - completion_loss: 7.4951 - problems_loss: 6.4886 - val_loss: 14.5267 - val_completion_loss: 8.5862 - val_problems_loss: 5.9406
Epoch 9/1000
3949/3949 [==============================] - 309s 78ms/step - loss: 13.9126 - completion_loss: 7.4522 - problems_loss: 6.4604 - val_loss: 14.4715 - val_completion_loss: 8.5681 - val_problems_loss: 5.9034
Epoch 10/1000
3949/3949 [==============================] - 306s 77ms/step - loss: 13.8571 - completion_loss: 7.4243 - problems_loss: 6.4328 - val_loss: 14.3032 - val_completion_loss: 8.3855 - val_problems_loss: 5.9177
Epoch 11/1000
3949/3949 [==============================] - 302s 77ms/step - loss: 13.7811 - completion_loss: 7.3670 - problems_loss: 6.4141 - va

Epoch 10/1000
3949/3949 [==============================] - 330s 84ms/step - loss: 14.4240 - completion_loss: 7.4141 - problems_loss: 7.0099 - val_loss: 14.1791 - val_completion_loss: 8.5092 - val_problems_loss: 5.6698
Epoch 11/1000
3949/3949 [==============================] - 321s 81ms/step - loss: 14.3727 - completion_loss: 7.3557 - problems_loss: 7.0169 - val_loss: 14.2311 - val_completion_loss: 8.5654 - val_problems_loss: 5.6657
Epoch 12/1000
3949/3949 [==============================] - 311s 79ms/step - loss: 14.3093 - completion_loss: 7.3323 - problems_loss: 6.9770 - val_loss: 14.1653 - val_completion_loss: 8.5244 - val_problems_loss: 5.6409
Epoch 13/1000
3949/3949 [==============================] - 298s 76ms/step - loss: 14.2649 - completion_loss: 7.2946 - problems_loss: 6.9703 - val_loss: 14.6635 - val_completion_loss: 8.9379 - val_problems_loss: 5.7257
Epoch 14/1000
3949/3949 [==============================] - 285s 72ms/step - loss: 14.2052 - completion_loss: 7.2592 - problems_l

In [5]:
# Load the data

exp_norm_map = pd.read_csv('data/experiment_information/exp_norm_map.csv')
results = pd.read_csv('attention_cross_validation_results.csv')

In [6]:
# Evaluate the results

norm_exp_dict = defaultdict(lambda: 'None')
for n, e in zip(exp_norm_map['normal_id'], exp_norm_map['experiment_id']):
    norm_exp_dict[n] = e
results['experiment_sequence'] = results['target_sequence'].map(norm_exp_dict)

metrics = []

# Calculate the metrics for each sequence
for sequence, df in results.groupby('target_sequence'):
    
    completion_target = df['completion_target']
    
    completion_prediction = df['completion_prediction']
    completion_auc = roc_auc_score(completion_target, completion_prediction) if len(completion_target.unique()) > 1 else None
    completion_acc = accuracy_score(completion_target, completion_prediction > 0.5)
    completion_r2 = r2_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_ev = explained_variance_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_mse = mean_squared_error(completion_target, completion_prediction)
    
    problems_target = df[df['completion_target'] == 1]['problems_target']
    problems_prediction = df[df['completion_target'] == 1]['problems_prediction']
    problems_mae = mean_absolute_error(problems_target, problems_prediction)
    problems_mape = mean_absolute_percentage_error(problems_target, problems_prediction)
    problems_r2 = r2_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_ev = explained_variance_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_mse = mean_squared_error(problems_target, problems_prediction)

    metrics.append([sequence, 
                    norm_exp_dict[sequence], 
                    len(df), 
                    completion_auc, 
                    completion_acc, 
                    completion_r2, 
                    completion_ev, 
                    completion_mse, 
                    problems_mae, 
                    problems_mape, 
                    problems_r2, 
                    problems_ev, 
                    problems_mse])

# Calculate the metrics for each fold
for fold, df in results.groupby('fold'):
    
    completion_target = df['completion_target']
    completion_prediction = df['completion_prediction']
    completion_auc = roc_auc_score(completion_target, completion_prediction) if len(completion_target.unique()) > 1 else None
    completion_acc = accuracy_score(completion_target, completion_prediction > 0.5)
    completion_r2 = r2_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_ev = explained_variance_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_mse = mean_squared_error(completion_target, completion_prediction)
    
    problems_target = df[df['completion_target'] == 1]['problems_target']
    problems_prediction = df[df['completion_target'] == 1]['problems_prediction']
    problems_mae = mean_absolute_error(problems_target, problems_prediction)
    problems_mape = mean_absolute_percentage_error(problems_target, problems_prediction)
    problems_r2 = r2_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_ev = explained_variance_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_mse = mean_squared_error(problems_target, problems_prediction)

    metrics.append([f'fold_{fold}', 
                    'None', 
                    len(df), 
                    completion_auc, 
                    completion_acc, 
                    completion_r2, 
                    completion_ev, 
                    completion_mse, 
                    problems_mae, 
                    problems_mape, 
                    problems_r2, 
                    problems_ev, 
                    problems_mse])

# Calculate the metrics for everything combined
completion_target = results['completion_target']
completion_prediction = results['completion_prediction']
completion_auc = roc_auc_score(completion_target, completion_prediction) if len(completion_target.unique()) > 1 else None
completion_acc = accuracy_score(completion_target, completion_prediction > 0.5)
completion_r2 = r2_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
completion_ev = explained_variance_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
completion_mse = mean_squared_error(completion_target, completion_prediction)

problems_target = results[results['completion_target'] == 1]['problems_target']
problems_prediction = results[results['completion_target'] == 1]['problems_prediction']
problems_mae = mean_absolute_error(problems_target, problems_prediction)
problems_mape = mean_absolute_percentage_error(problems_target, problems_prediction)
problems_r2 = r2_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
problems_ev = explained_variance_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
problems_mse = mean_squared_error(problems_target, problems_prediction)

metrics.append(['all_target_data', 
                'None', 
                len(results), 
                completion_auc, 
                completion_acc, 
                completion_r2, 
                completion_ev, 
                completion_mse, 
                problems_mae, 
                problems_mape, 
                problems_r2, 
                problems_ev, 
                problems_mse])

metrics = pd.DataFrame(metrics, 
                       columns=['sequence_id', 
                                'experiment_id', 
                                'sample_size', 
                                'completion_auc', 
                                'completion_acc', 
                                'completion_r2', 
                                'completion_ev', 
                                'completion_mse', 
                                'problems_mae', 
                                'problems_mape', 
                                'problems_r2', 
                                'problems_ev', 
                                'problems_mse'])

metrics.to_csv('attention_cross_validation_metrics.csv', index=False)

In [7]:
metrics.iloc[-6:]

,sequence_id,experiment_id,sample_size,completion_auc,completion_acc,completion_r2,completion_ev,completion_mse,problems_mae,problems_mape,problems_r2,problems_ev,problems_mse
288,fold_0,None,42114,0.761488,0.770599,0.163206,0.163224,0.156665,1.909619,0.186166,0.008309,0.009430,7.507026
289,fold_1,None,42114,0.766168,0.772498,0.174440,0.176006,0.157161,1.962280,0.190845,0.006882,0.008472,9.476802
290,fold_2,None,42114,0.764589,0.770551,0.169309,0.169693,0.157549,1.962772,0.178658,0.006852,0.010440,10.172661
291,fold_3,None,42113,0.752869,0.763137,0.149729,0.152107,0.163317,1.932924,0.193397,0.007455,0.008276,7.826903
292,fold_4,None,42113,0.759350,0.769169,0.165080,0.165093,0.158820,1.942819,0.187349,0.011808,0.013281,8.041776
293,all_target_data,None,210568,0.759990,0.769191,0.164371,0.164559,0.158702,1.942044,0.187272,0.008301,0.009944,8.604463
